In [ ]:

import os
from google.colab import files

# Upload kaggle.json
uploaded = files.upload()

# Create .kaggle directory if it doesn't exist
kaggle_dir = "/root/.kaggle"
os.makedirs(kaggle_dir, exist_ok=True)

# Move kaggle.json to the directory
kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")
with open("kaggle.json", "rb") as src:
    with open(kaggle_json_path, "wb") as dst:
        dst.write(src.read())

# Set permissions
os.chmod(kaggle_json_path, 0o600)

print("✅ kaggle.json uploaded, moved, and permissions set.")



In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saeedazfar/customized-cotton-disease-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.92G/1.92G [00:15<00:00, 132MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saeedazfar/customized-cotton-disease-dataset/versions/1


In [ ]:
import shutil
import os

# Source path (where kagglehub downloaded the dataset)
source_path = "/root/.cache/kagglehub/datasets/saeedazfar/customized-cotton-disease-dataset/versions/1"

# Destination path (more accessible in Colab)
destination_path = "/content/cotton_dataset"

# Copy the dataset folder
shutil.copytree(source_path, destination_path, dirs_exist_ok=True)

print("Dataset copied to:", destination_path)
print("Files:", os.listdir(destination_path))

Dataset copied to: /content/cotton_dataset
Files: ['Customized Cotton Dataset-Complete', 'Cotton-Disease-Training', 'Cotton-Disease-Validation']


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import kagglehub

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 1e-4
image_size = 224

# Data transforms
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder("/content/cotton_dataset/Cotton-Disease-Training/trainning/Cotton leaves - Training/800 Images", transform=transform)
val_dataset = datasets.ImageFolder("/content/cotton_dataset/Cotton-Disease-Validation/validation/Cotton plant disease-Validation/Cotton plant disease-Validation", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)




In [ ]:
# Load pretrained ResNet50
model = models.resnet50(pretrained=True)

# Modify final layer
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss:.4f} Train Acc: {train_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 194MB/s]


Epoch [1/10] Loss: 47.0654 Train Acc: 93.60%
Epoch [2/10] Loss: 12.9690 Train Acc: 98.11%
Epoch [3/10] Loss: 9.2952 Train Acc: 98.67%
Epoch [4/10] Loss: 8.4464 Train Acc: 98.82%
Epoch [5/10] Loss: 6.2697 Train Acc: 99.09%
Epoch [6/10] Loss: 11.9075 Train Acc: 98.13%
Epoch [7/10] Loss: 8.9262 Train Acc: 98.70%
Epoch [8/10] Loss: 5.3197 Train Acc: 99.28%
Epoch [9/10] Loss: 5.1332 Train Acc: 99.44%
Epoch [10/10] Loss: 5.6253 Train Acc: 99.25%


In [ ]:
model.eval()
val_correct = 0
val_total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        val_total += labels.size(0)
        val_correct += predicted.eq(labels).sum().item()

val_acc = 100 * val_correct / val_total
print(f"Validation Accuracy: {val_acc:.2f}%")
model.train()


Validation Accuracy: 96.08%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Save model
torch.save(model.state_dict(), "resnet50_wheat_disease.pth")
print("✅ Model training complete and saved.")

✅ Model training complete and saved.


In [ ]:
torch.save(model.state_dict(), "resnet50_cotton_disease.pth")


In [ ]:
from google.colab import files
files.download("resnet50_cotton_disease.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>